In [29]:
import cPickle
import gzip
import random
import numpy as np

def load_data(filename="../data/mnist.pkl.gz"):
    f = gzip.open(filename, 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorize(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

def vectorize(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

class QuadraticCost(object):
    
    @staticmethod
    def fn(a, y):
        return 0.5*np.linalg.norm(a-y)**2
    
    @staticmethod
    def delta(z, a, y):
        return (a-y) * sigmoid_prime(z)

class CrossEntropyCost(object):
    
    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    
    @staticmethod
    def delta(z, a, y):
        return (a-y)

class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        self.size = len(sizes)
        self.layers = sizes
        self.default_weight_initializer()
        self.cost=cost
        
    def default_weight_initializer(self):
        self.bias = [np.random.randn(y, 1) for y in self.layers[1:]]
        self.weight = [np.random.randn(y, x)/np.sqrt(x)
                       for x, y in zip(self.layers[:-1], self.layers[1:])]

    def large_weight_initializer(self):
        self.bias = [np.random.randn(y, 1) for y in self.layers[1:]]
        self.weight = [np.random.randn(y, x)
                       for x, y in zip(self.layers[:-1], self.layers[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.bias, self.weight):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0, test_data=None):
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                mini_batch = zip(*mini_batch)
                mini_batch_x = np.column_stack(mini_batch[0])
                mini_batch_y = np.column_stack(mini_batch[1])
                self.update_mini_batch((mini_batch_x, mini_batch_y), eta, lmbda, n)
            print(str(self.evaluate(test_data))+" / "+str(len(test_data)))


    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        mini_batch_x = mini_batch[0]
        mini_batch_y = mini_batch[1]
       
        gradient_w, gradient_b = self.backprop(mini_batch_x, mini_batch_y)
        self.weight = [(1-eta*(lmbda/n))*w-eta*nw
                        for w, nw in zip(self.weight, gradient_w)]
        self.bias = [b-eta*nb
                        for b, nb in zip(self.bias, gradient_b)]

    def backprop(self, x, y):
        gradient_w = [np.zeros(w.shape) for w in self.weight]
        gradient_b = [np.zeros(b.shape) for b in self.bias]
        batch_size = len(x[0])
        activations = [x]
        zs = []
        for w, b in zip(self.weight, self.bias):
            z = np.dot(w, x) + b
            zs.append(z)
            x = sigmoid(z)
            activations.append(x)

        deltaL = (self.cost).delta(zs[-1], x, y)
        gradient_b[-1] = np.sum(deltaL, axis=1, keepdims=True) * 1.0 / batch_size
        tmp_gradient_w = np.dot(deltaL, activations[-2].transpose())
        gradient_w[-1] = tmp_gradient_w * 1.0 / batch_size

        for l in xrange(2, self.size):
            deltaL = np.dot(self.weight[-l+1].transpose(), deltaL) * sigmoid_prime(zs[-l])
            gradient_b[-l] = np.sum(deltaL, axis=1, keepdims=True) * 1.0 / batch_size
            tmp_gradient_w = np.dot(deltaL, activations[-l-1].transpose())
            gradient_w[-l] =  tmp_gradient_w * 1.0 / batch_size

        return (gradient_w, gradient_b)


    def evaluate(self, test_data):
        result = [(np.argmax(self.feedforward(x)), y)
                    for x, y in test_data]
        return sum(int(x == y) for x, y in result)
    
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, 100, 10])
net.SGD(training_data, 60, 10, 0.1, lmbda=5.0, test_data=test_data)


9295 / 10000
9411 / 10000
9535 / 10000
9580 / 10000
9616 / 10000
9654 / 10000
9678 / 10000
9685 / 10000
9698 / 10000
9702 / 10000
9714 / 10000
9727 / 10000
9735 / 10000
9729 / 10000
9723 / 10000
9738 / 10000
9714 / 10000
9741 / 10000
9754 / 10000
9740 / 10000
9745 / 10000
9770 / 10000
9768 / 10000
9761 / 10000
9767 / 10000
9744 / 10000
9766 / 10000
9769 / 10000
9772 / 10000
9756 / 10000
9773 / 10000
9771 / 10000
9774 / 10000
9777 / 10000
9769 / 10000
9785 / 10000
9784 / 10000
9785 / 10000
9772 / 10000
9776 / 10000
9784 / 10000
9790 / 10000
9790 / 10000
9774 / 10000
9776 / 10000
9791 / 10000
9789 / 10000
9796 / 10000
9795 / 10000
9783 / 10000
9795 / 10000
9795 / 10000
9796 / 10000
9776 / 10000
9788 / 10000
9788 / 10000
9778 / 10000
9798 / 10000
9802 / 10000
9797 / 10000


# ====================================

In [30]:
"""network2.py
~~~~~~~~~~~~~~

An improved version of network.py, implementing the stochastic
gradient descent learning algorithm for a feedforward neural network.
Improvements include the addition of the cross-entropy cost function,
regularization, and better initialization of network weights.  Note
that I have focused on making the code simple, easily readable, and
easily modifiable.  It is not optimized, and omits many desirable
features.

"""

#### Libraries
# Standard library
import json
import random
import sys

# Third-party libraries
import numpy as np
import mnist_loader


#### Define the quadratic and cross-entropy cost functions

class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)


class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).

        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.

        """
        return (a-y)


#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        """The list ``sizes`` contains the number of neurons in the respective
        layers of the network.  For example, if the list was [2, 3, 1]
        then it would be a three-layer network, with the first layer
        containing 2 neurons, the second layer 3 neurons, and the
        third layer 1 neuron.  The biases and weights for the network
        are initialized randomly, using
        ``self.default_weight_initializer`` (see docstring for that
        method).

        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        """Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 over the square root of the number of
        weights connecting to the same neuron.  Initialize the biases
        using a Gaussian distribution with mean 0 and standard
        deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1.  Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        This weight and bias initializer uses the same approach as in
        Chapter 1, and is included for purposes of comparison.  It
        will usually be better to use the default weight initializer
        instead.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print "Epoch %s training complete" % j
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print "Cost on training data: {}".format(cost)
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print "Accuracy on training data: {} / {}".format(
                    accuracy, n)
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print "Cost on evaluation data: {}".format(cost)
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print "Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data)
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag ``convert`` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results ``y`` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in
        mnist_loader.load_data_wrapper.

        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()

#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net

#### Miscellaneous functions
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = Network([784, 100, 10])
net.SGD(training_data, 60, 10, 0.1,
        evaluation_data=test_data, lmbda = 5.0,
        monitor_evaluation_accuracy=True, monitor_training_accuracy=True)

Epoch 0 training complete
Accuracy on training data: 46405 / 50000
Accuracy on evaluation data: 9300 / 10000

Epoch 1 training complete
Accuracy on training data: 47288 / 50000
Accuracy on evaluation data: 9430 / 10000

Epoch 2 training complete
Accuracy on training data: 47841 / 50000
Accuracy on evaluation data: 9549 / 10000

Epoch 3 training complete
Accuracy on training data: 48178 / 50000
Accuracy on evaluation data: 9599 / 10000

Epoch 4 training complete
Accuracy on training data: 48372 / 50000
Accuracy on evaluation data: 9612 / 10000

Epoch 5 training complete
Accuracy on training data: 48568 / 50000
Accuracy on evaluation data: 9635 / 10000

Epoch 6 training complete
Accuracy on training data: 48602 / 50000
Accuracy on evaluation data: 9641 / 10000

Epoch 7 training complete
Accuracy on training data: 48702 / 50000
Accuracy on evaluation data: 9677 / 10000

Epoch 8 training complete
Accuracy on training data: 48939 / 50000
Accuracy on evaluation data: 9695 / 10000

Epoch 9 tr

([],
 [9300,
  9430,
  9549,
  9599,
  9612,
  9635,
  9641,
  9677,
  9695,
  9729,
  9718,
  9720,
  9725,
  9729,
  9732,
  9748,
  9739,
  9754,
  9748,
  9749,
  9747,
  9760,
  9757,
  9749,
  9759,
  9757,
  9773,
  9765,
  9762,
  9759,
  9759,
  9764,
  9765,
  9771,
  9764,
  9774,
  9779,
  9758,
  9775,
  9771,
  9783,
  9766,
  9780,
  9760,
  9782,
  9781,
  9775,
  9774,
  9793,
  9783,
  9767,
  9779,
  9783,
  9781,
  9782,
  9780,
  9784,
  9786,
  9784,
  9780],
 [],
 [46405,
  47288,
  47841,
  48178,
  48372,
  48568,
  48602,
  48702,
  48939,
  49069,
  49089,
  49153,
  49218,
  49205,
  49269,
  49303,
  49362,
  49393,
  49404,
  49441,
  49449,
  49469,
  49481,
  49444,
  49496,
  49519,
  49578,
  49553,
  49570,
  49512,
  49555,
  49571,
  49589,
  49614,
  49614,
  49611,
  49597,
  49579,
  49650,
  49629,
  49661,
  49640,
  49674,
  49588,
  49648,
  49669,
  49658,
  49683,
  49686,
  49688,
  49644,
  49651,
  49695,
  49688,
  49703,
  49662,
  496